In [7]:
# Genetic Algorithm With parameters
# Note this if checking if the optimisation still works with larger range parameters
import numpy as np

# Parameters
population_size = 10000   # Number of individuals in each generation
num_variables = 5        # Number of parameters to optimize
num_parents = 5000        # Number of parents selected to breed
num_generations = 10000   # Number of generations
crossover_rate = 0.8     # Probability of crossover
mutation_rate = 0.1      # Probability of mutation
mutation_range = [1, 5E5, 0.01, 1E-7, 5E-7, 5E-7]       # Range of mutation 
lower_bounds = [10, 50E6, 0.1, 0.5E-6, 10E-6, 50E-6]    # Bounds on parameters
upper_bounds = [1001, 500E6, 0.21, 500E-6, 500E-6, 500E-6]

# Test Solution
solution = np.array([55, 100E6, 0.5, 200E-6, 50E-6, 90E-6])

# Initialize population
no_pulses = np.random.randint(10, 1001, (population_size, 1))
f_0 = np.random.randint(50E6, 500E6, (population_size, 1))
amplitude = np.random.uniform(0.1, 0.21, (population_size, 1))
freq_sweep = np.random.uniform(0.5E-6, 500E-6, (population_size, 1))
duration = np.random.uniform(10E-6, 500E-6, (population_size, 1))
delay = np.random.uniform(50E-6, 500E-6, (population_size, 1))

max_values = np.array([1000, 500E6, 0.21, 500E-6, 500E-6, 500E-6])

population = np.hstack((no_pulses, f_0, amplitude, freq_sweep, duration, delay))

# Note new fitness that wil work with new parameters
def calculate_fitness(population, solution, max_values):
    # Normalize the solution and population using the max_values
    normalized_solution = solution / max_values
    normalized_population = population / max_values
    
    # Calculate the difference
    difference = 1 - np.abs(normalized_solution - normalized_population)
    
    # Calculate fitness as the mean of the differences
    fitness = np.mean(difference, axis=1)

    return fitness

def crossover(parents, crossover_rate, population_size):
    num_parents = parents.shape[0]
    num_variables = parents.shape[1]

    # Initialize offspring population array
    offspring_population = np.zeros((population_size, num_variables))
    current_count = 0

    while current_count < population_size:
        # Randomly shuffle the order of parents
        shuffled_indices = np.random.permutation(num_parents)
        shuffled_parents = parents[shuffled_indices, :]

        # Iterate over pairs of parents
        for i in range(0, num_parents, 2):
            if current_count >= population_size:
                break

            if i == num_parents - 1:
                offspring_population[current_count, :] = shuffled_parents[i, :]  # if odd number parents
                current_count += 1
            else:
                parent1 = shuffled_parents[i, :]
                parent2 = shuffled_parents[i + 1, :]

                if np.random.rand() < crossover_rate:
                    # Perform crossover
                    crossover_point = np.random.randint(1, num_variables)
                    offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                    offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
                else:
                    # No crossover, offspring are copies of parents
                    offspring1 = parent1
                    offspring2 = parent2

                # Add offspring to the population
                offspring_population[current_count, :] = offspring1
                current_count += 1

                if current_count < population_size:
                    offspring_population[current_count, :] = offspring2
                    current_count += 1

    return offspring_population

def mutate(new_population, mutation_rate, mutation_range):
    population_size = new_population.shape[0]
    num_variables = new_population.shape[1]

    mutated_population = new_population.copy()

    for i in range(population_size):
        for j in range(num_variables):
            if np.random.rand() < mutation_rate:
                mutation_value = (np.random.rand() - 0.5) * 2 * mutation_range[j]
                mutated_population[i, j] = mutated_population[i, j] + mutation_value
                mutated_population[i, j] = max(lower_bounds[j], min(upper_bounds[j], mutated_population[i, j]))

    return mutated_population

# Genetic Algorithm
for i in range(num_generations):
    # Fitness
    fitness = calculate_fitness(population, solution, max_values)

    # Selection
    parents = population[np.random.choice(population_size, num_parents, p=fitness/fitness.sum())]

    # Breeding Crossover and Mutation
    population = crossover(parents, crossover_rate, population_size)
    population = mutate(population, mutation_rate, mutation_range)

# Evaluation
fitness = calculate_fitness(population, solution, max_values)
max_value = np.max(fitness)
index_of_max = np.argmax(fitness)
min_value = np.min(fitness)
index_of_min = np.argmin(fitness)
avg_fit = np.mean(fitness)
best_fit = max_value
best = population[index_of_max, :]
worse_fit = min_value
worse = population[index_of_min, :]

print("Average Fitness:", avg_fit)
print("Best Fitness:", best_fit)
print("solution", solution)
print("Best Solution:", best)
print("Worst Fitness:", worse_fit)
print("Worst Solution:", worse)


Average Fitness: 0.7600561903450244
Best Fitness: 0.7693913154877866
solution [5.5e+01 1.0e+08 5.0e-01 2.0e-04 5.0e-05 9.0e-05]
Best Solution: [5.54844393e+01 1.00571532e+08 2.09963172e-01 2.00043818e-04
 4.96310485e-05 9.00356563e-05]
Worst Fitness: 0.7202253308167261
Worst Solution: [5.96108867e+01 1.02900377e+08 1.51903173e-01 2.00362037e-04
 5.20509170e-05 8.70967368e-05]
